In [1]:
import os, sys
import pandas as pd
from PIL import Image

from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split

from torchvision import transforms, models
from torchvision.transforms import Resize, ToTensor, Normalize

In [2]:
"""
common variable setting
* train_dir: directory of train dataset
* train_df: pd.DataFrame from 'train.csv'

* new_train_path: new path for revised .csv file
* new_train_df: pd.DataFrame for 'new_train.csv'
"""
train_dir = '/opt/ml/input/data/train'
train_df = pd.read_csv(os.path.join(train_dir, 'train.csv'))

new_train_path = os.path.join(train_dir, 'new_train.csv')
new_train_df = pd.DataFrame(None, columns=['path', 'label'])

batch_size = 10
num_epoch = 5
learning_rate = 0.0001

In [3]:
# train.csv를 수정해서 new_train.csv로 저장, 시간이 조금 걸립니다.
def cal_label(gender, age, mask):
    def cal_age(n):
        if n < 30:
            return 0
        elif n >= 60:
            return 2
        else:
            return 1
        
    _gender = 0 if gender == 'male' else 1
    _age = cal_age(age)
    _mask = {'incorrect_mask': 1, 
             'normal': 2
            }.get(mask, 0)
    
    return _mask * 6 + _gender * 3 + _age

for row in train_df.itertuples():
    img_dir = os.path.join(train_dir, 'images', row.path)
    img_names = [file for file in os.listdir(img_dir) if not file.startswith('.')]
    
    for name in img_names:
        data = {
            'path': os.path.join(img_dir, name),
            'label': cal_label(row.gender, row.age, name.split('.')[0])
        }
        
        new_train_df = new_train_df.append(data, ignore_index=True)

new_train_df.to_csv(new_train_path, index=False)
new_train_df.head()

,path,label
0,/opt/ml/input/data/train/images/000001_female_...,10
1,/opt/ml/input/data/train/images/000001_female_...,4
2,/opt/ml/input/data/train/images/000001_female_...,4
3,/opt/ml/input/data/train/images/000001_female_...,4
4,/opt/ml/input/data/train/images/000001_female_...,4


In [4]:
# Custom mask dataset
class MaskDataset(Dataset):
    def __init__(self, df, transform=transforms.Compose([ToTensor()])):
        self.img_paths = df.path.tolist()
        self.labels = df.label.tolist()
        self.transform = transform

    def __getitem__(self, index):
        X = Image.open(self.img_paths[index])
        Y = self.labels[index]

        return self.transform(X), Y

    def __len__(self):
        return len(self.img_paths)

In [5]:
# datset, dataloader setting
transform = transforms.Compose([
    Resize((512, 384), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
])

dataset = MaskDataset(new_train_df, transform=transform)
train_len = int(len(dataset) * 0.8)
valid_len = len(dataset) - train_len

train_dataset, valid_dataset = random_split(dataset, [train_len, valid_len])

dataloader = {
    'train': DataLoader(train_dataset, batch_size=batch_size, shuffle=True),
    'val': DataLoader(valid_dataset, shuffle=False)
}


print(f"Dataset size\n  train: {len(train_dataset)}\n  valid: {len(valid_dataset)}\n")
imgs, labels = next(iter(dataloader['train']))
print(f"Dataloader size\n  X: {imgs.size()}\n  Y: {labels.size()}")

Dataset size
  train: 15120
  valid: 3780

Dataloader size
  X: torch.Size([10, 3, 512, 384])
  Y: torch.Size([10])


In [6]:
# model setting
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 18)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [7]:
# train parameter setting
device = torch.device('cuda')

model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [8]:
# train process
for i in range(1, num_epoch + 1):
    ##### train ####
    model.train()
    train_loss = 0.
    train_accu = 0.
    for imgs, labels in tqdm(dataloader['train']):
        imgs, labels = imgs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        logit = model(imgs)        
        loss = criterion(logit, labels)
        loss.backward()
        optimizer.step()
        
        _, preds = logit.max(dim=1)
        train_loss += loss.item()
        train_accu += torch.sum(preds == labels)
    
    
    #### validation ####
    model.eval()
    val_loss = 0.
    val_accu = 0.
    for imgs, labels in dataloader['val']:
        imgs, labels = imgs.to(device), labels.to(device)
        
        logit = model(imgs)
        loss = criterion(logit, labels)
        
        _, preds = logit.max(dim=1)
        val_loss += loss.item()
        val_accu += torch.sum(preds == labels)
    
    print(f'Epoch {i}/{num_epoch}')
    print('-' * 10)
    print(f"train_loss: {train_loss / len(dataloader['train'].dataset):.4f}\ttrain_accuracy: {train_accu / len(dataloader['train'].dataset):.4f}")
    print(f"val_loss: {val_loss / len(dataloader['val'].dataset):.4f}\tval_accuracy: {val_accu / len(dataloader['val'].dataset):.4f}")
    print()

  0%|          | 0/1512 [00:00<?, ?it/s]

Epoch 1/5
----------


  0%|          | 1/1512 [00:00<03:35,  7.00it/s]

train_loss: 0.0477	train_accuracy: 0.8614
val_loss: 0.1896	val_accuracy: 0.9407

Epoch 2/5
----------


  0%|          | 1/1512 [00:00<03:40,  6.84it/s]

...

train_loss: 0.0074	train_accuracy: 0.9790
val_loss: 0.1308	val_accuracy: 0.9574

Epoch 4/5
----------


  0%|          | 1/1512 [00:00<03:30,  7.16it/s]

train_loss: 0.0051	train_accuracy: 0.9852
val_loss: 0.0465	val_accuracy: 0.9892

Epoch 5/5
----------


100%|██████████| 1512/1512 [03:22<00:00,  7.45it/s]


train_loss: 0.0038	train_accuracy: 0.9886
val_loss: 0.1044	val_accuracy: 0.9685



In [9]:
# save model
torch.save(model.state_dict(), f'./resnet18_epoch{num_epoch}.pth')